In [1]:
%cd ..

/home/devel/notwork/reface-fake-detection


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torchvision 

from efficientnet_pytorch import EfficientNet

In [7]:
class CustomModel(torch.nn.Module):
    def __init__(self, model_name="efficientnet-b0", in_channels=12, num_classes=2):
        super().__init__()
        
        self.encoder = EfficientNet.from_pretrained(model_name, in_channels=3)
        layers_to_remove = ["_fc", "_swish"]
        for l in layers_to_remove:
            setattr(self.encoder, l, torch.nn.Identity())
        
        self.in_images = in_channels // 3
        self.head = torch.nn.Sequential(
            torch.nn.Linear(self.in_images * 1280, 1280),
            torch.nn.BatchNorm1d(1280),
            torch.nn.ReLU(),
            torch.nn.Linear(1280, 128),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, num_classes)
        )
        
    def forward(self, x):
        encoded = []
        for i in range(self.in_images):
            img = x[:, i*3:i*3+3, :, :]
            encoded.append(self.encoder(img))
            
        features = torch.cat(encoded, 1)
        out = self.head(features)
        return out
    
model = CustomModel()

Loaded pretrained weights for efficientnet-b0


In [6]:
model(torch.rand(64, 12, 256, 256)).shape

torch.Size([64, 1280])
torch.Size([64, 5120])


torch.Size([64, 2])

In [29]:
m = EfficientNet.from_pretrained("efficientnet-b0")

Loaded pretrained weights for efficientnet-b0


In [37]:
setattr(m, "_swish", torch.nn.Identity())

In [ ]:
EfficientNet.from_pretrained("efficientnet-b0").half